In [1]:
library("data.table")
library("qtl")

Warning message:
“package ‘data.table’ was built under R version 3.5.2”Warning message:
“package ‘qtl’ was built under R version 3.5.2”

In [2]:
datf2 <- read.cross(format = "csvs",
                    genotypes = c(-1,0,1,"C","D"),
                    dir = "/home/tilman/optimise_test/Stripes/stripes2rqtl/home/optimise_test/str2qtl_out/F2",
                    genfile="F2_rqtl_genotypes_ws200_FilteredForBinsize1000000_FilteredForXO3removedAmb0.2.csv",
                    phefile = "F2_rqtl_phenotypes_ws200_FilteredForBinsize_FilteredForXO3removedAmb0.2.csv" )

Warning message in file(file, "rt"):
“cannot open file '/home/tilman/optimise_test/Stripes/stripes2rqtl/home/optimise_test/str2qtl_out/F2/F2_rqtl_genotypes_ws200_FilteredForBinsize1000000_FilteredForXO3removedAmb0.2.csv': No such file or directory”

ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
summary(datf2)

In [ ]:
plotMissing(datf2)

In [ ]:
par(mfrow=c(1,2), las=1)
plot(ntyped(datf2), ylab="No. typed markers", main="No. genotypes by individual")
plot(ntyped(datf2, "mar"), ylab="No. typed individuals", main="No. genotypes by marker")

In [ ]:
nt.bymar <- ntyped(datf2, "mar")
todrop <- names(nt.bymar[nt.bymar < 450]) # less than 50% of all markers


In [ ]:
todrop

In [ ]:
datf2.1 <- drop.markers(datf2, todrop)

In [ ]:
par(mfrow=c(1,2), las=1)
plot(ntyped(datf2.1), ylab="No. typed markers", main="No. genotypes by individual") 
plot(ntyped(datf2.1, "mar"), ylab="No. typed individuals",main="No. genotypes by marker")

In [ ]:
datf2.2 <- subset(datf2.1, ind=(ntyped(datf2.1)>100)) # removing all individuals that have less than 10% of all markers


In [ ]:
length(ntyped(datf2.1, "ind"))
length(ntyped(datf2.2, "ind"))

In [ ]:
par(mfrow=c(1,2), las=1)
plot(ntyped(datf2.2), ylab="No. typed markers", main="No. genotypes by individual") 
plot(ntyped(datf2.2, "mar"), ylab="No. typed individuals",main="No. genotypes by marker")

In [ ]:
summary(datf2.2)

In [ ]:
cg <- comparegeno(datf2.2)

In [ ]:
hist(cg[lower.tri(cg)], breaks=seq(0, 1, len=101), xlab="No. matching genotypes") 
rug(cg[lower.tri(cg)])

In [ ]:
wh <- which(cg > 0.9, arr=TRUE)
wh <- wh[wh[,1] < wh[,2],]

In [ ]:
wh

In [ ]:
print(dup <- findDupMarkers(datf2.2, exact.only=FALSE))

In [ ]:
gt <- geno.table(datf2.2)

In [ ]:
gt[gt$P.value < 0.05/totmar(datf2.2),]
todrop <- rownames(gt[gt$P.value < 0.05/totmar(datf2.2),])
datf2.3 <- drop.markers(datf2.2, todrop)

In [ ]:
length(todrop)

In [ ]:
datf2.3 <- datf2.2

In [ ]:
g <- pull.geno(datf2.2)
gfreq <- apply(g, 1, function(a) table(factor(a, levels=1:3)))
gfreq <- t(t(gfreq) / colSums(gfreq))
par(mfrow=c(1,3), las=1)
for(i in 1:3) plot(gfreq[i,], ylab="Genotype frequency", main=c("AA", "AB", "BB")[i], ylim=c(0,1))

In [ ]:
g <- pull.geno(datf2.3)
gfreq <- apply(g, 1, function(a) table(factor(a, levels=1:3)))
gfreq <- t(t(gfreq) / colSums(gfreq))
par(mfrow=c(1,3), las=1)
for(i in 1:3) plot(gfreq[i,], ylab="Genotype frequency", main=c("AA", "AB", "BB")[i], ylim=c(0,1))

In [ ]:
id_gtfreq <- which(gfreq[1,]<0.5 & gfreq[2,]<0.65 & gfreq[2,]>0.35 & gfreq[3,]<0.5)

In [ ]:
datf2.4 <- subset(datf2.3, ind=(id_gtfreq))

In [ ]:
datf2.4 <- datf2.3

In [ ]:
g <- pull.geno(datf2.4)
gfreq <- apply(g, 1, function(a) table(factor(a, levels=1:3)))
gfreq <- t(t(gfreq) / colSums(gfreq))
par(mfrow=c(1,3), las=1)
for(i in 1:3) plot(gfreq[i,], ylab="Genotype frequency", main=c("AA", "AB", "BB")[i], ylim=c(0,1))

In [ ]:
datf2.4 <- est.rf(datf2.4)
datf2.4_tst <- markerlrt(datf2.4)

In [ ]:
rf <- pull.rf(datf2.4)
lod <- pull.rf(datf2.4, what="lod")
plot(as.numeric(rf), as.numeric(lod), xlab="Recombination fraction", ylab="LOD score")

In [ ]:
plotRF(datf2.4, alternate.chrid=TRUE)

In [ ]:
genmap <- est.map(datf2.4, error.prob=0.005)

In [ ]:
summaryMap(genmap)

In [ ]:
plotMap(genmap)

In [ ]:
genmap

In [ ]:
datf2.4 <- replace.map(datf2.4, genmap)

In [ ]:
plot(countXO(datf2.4), ylab="Number of crossovers")
hist(countXO(datf2.4), ylab="Number of crossovers")
mean(countXO(datf2.4), ylab="Number of crossovers")


In [ ]:
datf2.5 <- datf2.4

In [ ]:
plotPheno(datf2.5, pheno.col =2 )

In [ ]:
#plotMissing(datf2.5, reorder=TRUE)

In [ ]:
datf2.5 <- calc.genoprob(datf2.5, step=1)

In [ ]:
datf2.5 <- calc.genoprob(datf2.5, step=0, off.end = 0)


In [ ]:
out.em <- scanone(datf2.5, pheno.col = 2)

In [ ]:
plot(out.em)

In [ ]:
plot(out.em,chr=1)

In [ ]:
ac <- pull.pheno(datf2.5, c("SEX"))

In [ ]:
ac2 <- pull.pheno(datf2.5, c("SEX","DAM","SIRE"))

In [ ]:
out.acovar <- scanone(datf2.5, pheno.col = 2, addcovar=ac2)

In [ ]:
plot(out.acovar, )

In [ ]:
plot(out.acovar, chr=1)

In [ ]:
out.acovar

In [ ]:
#write.csv(out.acovar, "20190828_out_f2_bw8_ctrlsex.csv")